In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys,os
import json
import random
import csv
import string
import swifter
import pymysql
import nltk
import ast
from nltk import tokenize

from gensim.parsing.preprocessing import preprocess_documents, preprocess_string, strip_tags, strip_punctuation, remove_stopwords, strip_numeric

In [2]:
csv.field_size_limit(100000000)
csv.field_size_limit()

100000000

In [20]:
CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, strip_numeric, remove_stopwords]
df = pd.DataFrame(columns=['id', 'type', 'title', 'preprocess'])
df.to_csv("../Data/cleaned_data.csv")
csv_iter = pd.read_csv('../Data/filtered_data.csv', iterator=True, chunksize=200000, dtype = {'type' : str, 'content' : str, 'title' : str}, encoding = 'utf-8', error_bad_lines = False, engine='python')
for single_df in csv_iter:
    single_df['preprocess'] = single_df['content'].swifter.apply(lambda x: preprocess_string(x, CUSTOM_FILTERS))
    with open('../Data/cleaned_data.csv', 'a') as f:
        single_df.to_csv(f, header=False, columns=['id', 'type', 'title', 'preprocess'])

Pandas Apply: 100%|██████████| 7968/7968 [00:03<00:00, 1994.14it/s]


In [3]:
df = pd.DataFrame(columns=['id', 'length', 'type'])
nlines = 50000
total = 50001
for i in np.arange(1, total, nlines):
    preprocess_data = pd.read_csv('../Data/cleaned_data.csv', converters={'preprocess':ast.literal_eval}, skiprows=i, nrows = nlines, header=None, names=['id', 'type', 'title', 'preprocess'])
    preprocess_data['length'] = preprocess_data['preprocess'].swifter.apply(lambda x: len(x))
    df = pd.concat([df, preprocess_data[['id', 'length', 'type']]])

Pandas Apply: 100%|██████████| 50000/50000 [00:00<00:00, 630889.38it/s]


In [4]:
length = np.array(df['length'].tolist())

In [5]:
np.percentile(length, 90)

662.0

In [93]:
text = preprocess_data.iloc[4]['preprocess']

In [94]:
text

['photo',
 'opinion',
 'writer',
 'age',
 'trump',
 'it’s',
 'question',
 'thought',
 'i’d',
 'share',
 'days’',
 'worth',
 'false',
 'starts',
 'ledes',
 'tuesday’s',
 'news',
 'north',
 'korea',
 'test',
 'fired',
 'ballistic',
 'missile',
 'hit',
 'washington',
 'd',
 'c',
 'started',
 'drafting',
 'piece',
 'opened',
 'follows',
 '“jim',
 'mattis',
 'familiar',
 'napoleon’s',
 'famous',
 'orders',
 'field',
 'marshals',
 '‘if',
 'start',
 'vienna',
 '—',
 'vienna',
 '’',
 'commander',
 'iraq',
 'mattis',
 'f',
 'bombed',
 'version',
 'line',
 'rage',
 'bush',
 'administration’s',
 'abrupt',
 'decision',
 'pull',
 'planned',
 'assault',
 'city',
 'falluja',
 'tough',
 'talk',
 'married',
 'visible',
 'hesitation',
 'worst',
 'approach',
 'possible',
 'enemy',
 '“but',
 'that’s',
 'good',
 'description',
 'trump',
 'administration’s',
 'policy',
 'pyongyang',
 'today',
 '…”',
 'came',
 'wednesday',
 'morning',
 'donald',
 'trump',
 'twitter',
 'fever',
 'reliving',
 'madness',
 'king

In [53]:
import gensim.downloader as API

In [54]:
model = API.load('word2vec-google-news-300')

In [113]:
vocab = model.vocab.keys()
def text2vec(x, max_length):
    vectors= np.array([])
    i = 0
    for w in x:
        i += 1
        if w in vocab:
            vectors = np.append(vectors, model[w], axis=0)
        else:
            vectors = np.append(vectors, np.zeros(300), axis=0)
    if i < max_length:
        for i in range(i, max_length):
            vectors = np.append(vectors, np.zeros(300), axis=0)
    elif i > max_length:
        vectors = vectors[0:max_length]
    return vectors.reshape(max_length, 300)

In [114]:
words = text2vec(text, 557)

In [116]:
words.shape

(557, 300)